In [1]:
import pandas as pd
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sys
import datetime
import os
import seaborn as sns
import math as mth
import pickle
import re
import nltk
import spacy
import sklearn

import tika
from tika import parser

import bokeh
from bokeh.plotting import *
from bokeh.io import output_notebook
from bokeh.models import Toggle, BoxAnnotation, CustomJS, HoverTool

output_notebook()

from sklearn.manifold import TSNE

sns.set_style("white")

%matplotlib inline

from IPython.display import display

from IPython.display import HTML

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Loading BokehJS ...

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zanderl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df1 = pd.read_csv('C:/Users/zanderl/Documents/Data Science/SFDC Similar Companies Project/SFDC and Ipreo CoData.csv',encoding='latin1')

In [4]:
df1.index = df1['Account Name']

df1.shape
df1.head(5)

(994, 64)

,Account Name,Business Description,Business Description Plain,Investment Approach,Billing Country,Billing State/Province,Billing City,Account ID (Case Safe),S&P Capital IQ Company ID,KeyInstn - SNL,...,Dominant Style,Dominant Orientation,Contact Equity Assets ($m),Account Equity Assets Under Management ($m),City,State,Country,Geographic Region,CRM,Primary Contacts
Account Name,,,,,,,,,,,,,,,,,,,,,
12 West Capital Management LP,12 West Capital Management LP,12 West Capital Management LP is an employee o...,12 West Capital Management LP is an employee o...,NaN,United States,NY,New York,0014000000diOsEAAU,IQ158358908,4321055.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1832 Asset Management L.P.,1832 Asset Management L.P.,1832 Asset Management L.P. is a privately owne...,1832 Asset Management L.P. is a privately owne...,NaN,Canada,ON,Toronto,0014000000FGy4YAAT,IQ1029378,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1919 Investment Counsel, LLC","1919 Investment Counsel, LLC","1919 Investment Counsel, LLC operates as a sub...","1919 Investment Counsel, LLC operates as a sub...",NaN,United States,MD,Baltimore,0014000001llGwcAAE,IQ59705013,4096352.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361° Capital LLC,361° Capital LLC,361° Capital LLC is an employee owned hedge fu...,361° Capital LLC is an employee owned hedge fu...,NaN,United States,CO,Denver,0014000000cCrlGAAS,IQ116742844,4117083.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"3G Capital, Inc.","3G Capital, Inc.","3G Capital, Inc. is a private equity firm spec...","3G Capital, Inc. is a private equity firm spec...",NaN,United States,NY,New York,0014000000K85RQAAZ,IQ23471434,4189265.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
coData_df = pd.DataFrame(df1,columns=['Business Description'],index=df1.index)

In [6]:
coData_df.head()

,Business Description
Account Name,
12 West Capital Management LP,12 West Capital Management LP is an employee o...
1832 Asset Management L.P.,1832 Asset Management L.P. is a privately owne...
"1919 Investment Counsel, LLC","1919 Investment Counsel, LLC operates as a sub..."
361° Capital LLC,361° Capital LLC is an employee owned hedge fu...
"3G Capital, Inc.","3G Capital, Inc. is a private equity firm spec..."


In [13]:
df1.loc['12 West Capital Management LP']['Business Description']

'12 west capital management lp is an employee owned hedge fund sponsor. it primarily provides its services to pooled investment vehicles. the firm launches and manages hedge fund for its clients. it invests in the public equity and fixed income markets across the globe. the firm invests in value stocks of small-cap companies. it employs a long/short equity strategy to make its investments. 12 west capital management lp is based in new york city. in managing the funds, we invest both long and short, in equity, equity related, and credit securities. in addition, we may also invest in preferred stocks, warrants, rights, options, swaps and other derivative instruments, bonds and other fixed income securities, futures and money market instruments. we believe that investing in companies with an equity market capitalization of u.s. $2.5 billion or less (\x93small cap\x94) offers the greatest opportunity to generate superior compounded annual net returns, predominantly because: (i) there is a 

In [14]:
df1['Business Description'] = df1['Business Description'].str.lower()
coData_df['Business Description'] = coData_df['Business Description'].str.lower()

In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import DBSCAN
import gensim
import nltk

In [16]:
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1,4),
    stop_words='english',
    min_df = 10,
    max_df = 0.60
    # Original value: max_df = 0.8
)

# 3/12/2018: Changed ngram_range=(1,2) to ngram_range(1,3)

In [17]:
%%time

X = vectorizer.fit_transform(coData_df['Business Description'])
X

Wall time: 935 ms


In [18]:
type(X)
X.shape

scipy.sparse.csr.csr_matrix

(994, 3562)

In [19]:
feat_names = vectorizer.get_feature_names()[:]
len(feat_names)

feat_names

3562

['10',
 '10 billion',
 '10 million',
 '100',
 '100 million',
 '1000',
 '1000 growth',
 '1000 growth index',
 '1000 value',
 '1000 value index',
 '1000 value index russell',
 '12',
 '15',
 '150',
 '150 million',
 '1969',
 '1969 based',
 '1971',
 '1971 based',
 '1972',
 '1972 based',
 '1980',
 '1980 based',
 '1982',
 '1982 based',
 '1983',
 '1983 based',
 '1984',
 '1984 based',
 '1985',
 '1985 based',
 '1986',
 '1986 based',
 '1987',
 '1987 based',
 '1988',
 '1988 based',
 '1989',
 '1989 based',
 '1990',
 '1990 based',
 '1991',
 '1991 based',
 '1992',
 '1992 based',
 '1993',
 '1993 based',
 '1994',
 '1994 based',
 '1995',
 '1995 based',
 '1996',
 '1996 based',
 '1997',
 '1997 based',
 '1998',
 '1998 based',
 '1999',
 '1999 based',
 '20',
 '20 million',
 '200',
 '200 million',
 '2000',
 '2000 based',
 '2000 growth',
 '2000 growth index',
 '2000 index',
 '2000 index russell',
 '2000 value',
 '2000 value index',
 '2001',
 '2001 based',
 '2002',
 '2002 based',
 '2003',
 '2003 based',
 '2004'